In [32]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn import svm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
drive.mount('/content/drive')

data = pd.read_csv('/content/drive/Shared drives/539 Project/final_data.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
data = data.drop(['Unnamed: 0'], axis = 1)
data
#only including data from 2014, as this is the year with major regulation changes to a hybrid engine, 
#the same engine used in current regulations

,season,round,circuit_id,date,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,driver,...,driver_points,driver_wins,driver_standings_pos,constructor_points_after_race,constructor_wins_after_race,constructor_standings_pos_after_race,constructor_points,constructor_wins,constructor_standings_pos,qualifying_time
0,2014,1,albert_park,2014-03-16,0,0,0,0,0,rosberg,...,0.0,0.0,0.0,25.0,1,2,0.0,0.0,0.0,01:44.6
1,2014,1,albert_park,2014-03-16,0,0,0,0,0,kevin_magnussen,...,0.0,0.0,0.0,33.0,0,1,0.0,0.0,0.0,01:45.7
2,2014,1,albert_park,2014-03-16,0,0,0,0,0,button,...,0.0,0.0,0.0,33.0,0,1,0.0,0.0,0.0,01:44.4
3,2014,1,albert_park,2014-03-16,0,0,0,0,0,alonso,...,0.0,0.0,0.0,18.0,0,3,0.0,0.0,0.0,01:45.8
4,2014,1,albert_park,2014-03-16,0,0,0,0,0,bottas,...,0.0,0.0,0.0,10.0,0,4,0.0,0.0,0.0,01:48.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3277,2022,4,imola,2022-04-24,0,0,0,1,1,latifi,...,0.0,0.0,20.0,1.0,0,10,1.0,0.0,9.0,01:22.3
3278,2022,4,imola,2022-04-24,0,0,0,1,1,mick_schumacher,...,0.0,0.0,17.0,15.0,0,8,12.0,0.0,7.0,01:19.0
3279,2022,4,imola,2022-04-24,0,0,0,1,1,ricciardo,...,8.0,0.0,11.0,46.0,0,4,24.0,0.0,4.0,01:29.7
3280,2022,4,imola,2022-04-24,0,0,0,1,1,alonso,...,2.0,0.0,14.0,22.0,0,6,22.0,0.0,5.0,01:31.1


In [34]:
#Pivoting the data so there are no strings, have columns for each track and constructor

In [35]:
# calculate age of drivers

from dateutil.relativedelta import *

data['date'] = pd.to_datetime(data.date)
data['date_of_birth'] = pd.to_datetime(data.date_of_birth)
data['driver_age'] = data.apply(lambda x: relativedelta(x['date'], x['date_of_birth']).years, axis=1)
data.drop(['date', 'date_of_birth'], axis = 1, inplace = True)

In [36]:
# fill/drop nulls

for col in ['driver_points', 'driver_wins', 'driver_standings_pos', 'constructor_points', 
            'constructor_wins' , 'constructor_standings_pos']:
    data[col].fillna(0, inplace = True)
    data[col] = data[col].map(lambda x: int(x))
    
data.dropna(inplace = True )

In [37]:
# convert to boolean

for col in ['weather_warm', 'weather_cold','weather_dry', 'weather_wet', 'weather_cloudy']:
    data[col] = data[col].map(lambda x: bool(x))

In [38]:
# calculate difference in qualifying times
final_df = data


final_df['qualifying_time'] = final_df.qualifying_time.map(lambda x: 0 if str(x) == '00.000' 
                             else(float(str(x).split(':')[1]) + (60 * float(str(x).split(':')[0])) if x != 0 or x==0 else 0))
final_df = final_df[final_df['qualifying_time'] != 0]
final_df.sort_values(['season', 'round', 'grid'], inplace = True)
final_df['qualifying_time_diff'] = final_df.groupby(['season', 'round']).qualifying_time.diff()
final_df['qualifying_time'] = final_df.groupby(['season', 'round']).qualifying_time_diff.cumsum().fillna(0)
final_df.drop('qualifying_time_diff', axis = 1, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [39]:
data = final_df
data

,season,round,circuit_id,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,driver,nationality,...,driver_wins,driver_standings_pos,constructor_points_after_race,constructor_wins_after_race,constructor_standings_pos_after_race,constructor_points,constructor_wins,constructor_standings_pos,qualifying_time,driver_age
17,2014,1,albert_park,False,False,False,False,False,hamilton,British,...,0,0,25.0,1,2,0,0,0,0.0,29
20,2014,1,albert_park,False,False,False,False,False,ricciardo,Australian,...,0,0,0.0,0,11,0,0,0,0.3,24
0,2014,1,albert_park,False,False,False,False,False,rosberg,German,...,0,0,25.0,1,2,0,0,0,0.4,28
1,2014,1,albert_park,False,False,False,False,False,kevin_magnussen,Danish,...,0,0,33.0,0,1,0,0,0,1.5,21
3,2014,1,albert_park,False,False,False,False,False,alonso,Spanish,...,0,0,18.0,0,3,0,0,0,1.6,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3272,2022,4,imola,False,False,False,True,True,stroll,Canadian,...,0,18,5.0,0,9,0,0,10,0.1,23
3276,2022,4,imola,False,False,False,True,True,ocon,French,...,0,7,22.0,0,6,22,0,5,-7.5,25
3274,2022,4,imola,False,False,False,True,True,gasly,French,...,0,12,16.0,0,7,10,0,8,-7.3,26
3273,2022,4,imola,False,False,False,True,True,albon,Thai,...,0,16,1.0,0,10,1,0,9,-6.0,26


In [40]:
# get dummies

df_dum = pd.get_dummies(final_df, columns = ['circuit_id', 'nationality', 'constructor'] )

for col in df_dum.columns:
    if 'nationality' in col and df_dum[col].sum() < 140:
        df_dum.drop(col, axis = 1, inplace = True)
        
    elif 'constructor' in col and df_dum[col].sum() < 140:
        df_dum.drop(col, axis = 1, inplace = True)
        
    elif 'circuit_id' in col and df_dum[col].sum() < 70:
        df_dum.drop(col, axis = 1, inplace = True)
    
    else:
        pass

In [41]:
df_dum.shape
#shape of final dataset

(3125, 61)

In [42]:
df_dum.to_csv('final_df.csv', index = False)

In [43]:
df = df_dum
df

,season,round,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,driver,grid,podium,...,constructor_ferrari,constructor_force_india,constructor_haas,constructor_mclaren,constructor_mercedes,constructor_red_bull,constructor_renault,constructor_sauber,constructor_toro_rosso,constructor_williams
17,2014,1,False,False,False,False,False,hamilton,1,19,...,0,0,0,0,1,0,0,0,0,0
20,2014,1,False,False,False,False,False,ricciardo,2,22,...,0,0,0,0,0,1,0,0,0,0
0,2014,1,False,False,False,False,False,rosberg,3,1,...,0,0,0,0,1,0,0,0,0,0
1,2014,1,False,False,False,False,False,kevin_magnussen,4,2,...,0,0,0,1,0,0,0,0,0,0
3,2014,1,False,False,False,False,False,alonso,5,4,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3272,2022,4,False,False,False,True,True,stroll,15,10,...,0,0,0,0,0,0,0,0,0,0
3276,2022,4,False,False,False,True,True,ocon,16,14,...,0,0,0,0,0,0,0,0,0,0
3274,2022,4,False,False,False,True,True,gasly,17,12,...,0,0,0,0,0,0,0,0,0,0
3273,2022,4,False,False,False,True,True,albon,18,11,...,0,0,0,0,0,0,0,0,0,1


In [44]:
def score_classification(model):
    score = 0
    for circuit in df[df.season == 2020]['round'].unique():

        test = df[(df.season == 2020) & (df['round'] == circuit)]
        X_test = test.drop(['driver', 'podium', 'constructor_points_after_race', 'constructor_wins_after_race', 'constructor_standings_pos_after_race'], axis = 1)
        y_test = test.podium

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns = ['proba_0', 'proba_1'])
        prediction_df['actual'] = y_test.reset_index(drop = True)
        prediction_df.sort_values('proba_1', ascending = False, inplace = True)
        prediction_df.reset_index(inplace = True, drop = True)
        prediction_df['predicted'] = prediction_df.index
        prediction_df['predicted'] = prediction_df.predicted.map(lambda x: 1 if x == 0 else 0)

        score += precision_score(prediction_df.actual, prediction_df.predicted)

    model_score = score / df[df.season == 2020]['round'].unique().max()
    return model_score

In [15]:
df = df.copy()
#this gives a y that is 1 if the driver won the race, otherwise 0, the response we are predicting is the winner, that is y=1
df.podium = df.podium.map(lambda x: 1 if x == 1 else 0)

train = df[df.season <2020]

X_train = train.drop(['driver', 'podium', 'constructor_points_after_race', 'constructor_wins_after_race', 'constructor_standings_pos_after_race'], axis = 1)
y_train = train.podium
scaler = StandardScaler()
X_train
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

In [26]:
X_train

,season,round,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,grid,driver_points,driver_wins,...,constructor_ferrari,constructor_force_india,constructor_haas,constructor_mclaren,constructor_mercedes,constructor_red_bull,constructor_renault,constructor_sauber,constructor_toro_rosso,constructor_williams
0,-1.488702,-1.636423,-0.550345,0.0,-0.303951,-0.271015,-0.457326,-1.618533,-0.678040,-0.320791,...,-0.333097,-0.301441,-0.266469,-0.325965,3.002127,-0.324367,-0.264635,-0.298073,-0.322764,-0.329939
1,-1.488702,-1.636423,-0.550345,0.0,-0.303951,-0.271015,-0.457326,-1.446316,-0.678040,-0.320791,...,-0.333097,-0.301441,-0.266469,-0.325965,-0.333097,3.082931,-0.264635,-0.298073,-0.322764,-0.329939
2,-1.488702,-1.636423,-0.550345,0.0,-0.303951,-0.271015,-0.457326,-1.274099,-0.678040,-0.320791,...,-0.333097,-0.301441,-0.266469,-0.325965,3.002127,-0.324367,-0.264635,-0.298073,-0.322764,-0.329939
3,-1.488702,-1.636423,-0.550345,0.0,-0.303951,-0.271015,-0.457326,-1.101882,-0.678040,-0.320791,...,-0.333097,-0.301441,-0.266469,3.067818,-0.333097,-0.324367,-0.264635,-0.298073,-0.322764,-0.329939
4,-1.488702,-1.636423,-0.550345,0.0,-0.303951,-0.271015,-0.457326,-0.929665,-0.678040,-0.320791,...,3.002127,-0.301441,-0.266469,-0.325965,-0.333097,-0.324367,-0.264635,-0.298073,-0.322764,-0.329939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2348,1.464853,1.777901,1.817043,0.0,-0.303951,-0.271015,-0.457326,0.964723,-0.480451,-0.320791,...,-0.333097,-0.301441,-0.266469,-0.325965,-0.333097,-0.324367,-0.264635,-0.298073,-0.322764,-0.329939
2349,1.464853,1.777901,1.817043,0.0,-0.303951,-0.271015,-0.457326,1.136940,-0.071161,-0.320791,...,-0.333097,-0.301441,-0.266469,-0.325965,-0.333097,-0.324367,-0.264635,-0.298073,-0.322764,-0.329939
2350,1.464853,1.777901,1.817043,0.0,-0.303951,-0.271015,-0.457326,1.309157,-0.678040,-0.320791,...,-0.333097,-0.301441,-0.266469,-0.325965,-0.333097,-0.324367,-0.264635,-0.298073,-0.322764,3.030866
2351,1.464853,1.777901,1.817043,0.0,-0.303951,-0.271015,-0.457326,1.481374,-0.663926,-0.320791,...,-0.333097,-0.301441,-0.266469,-0.325965,-0.333097,-0.324367,-0.264635,-0.298073,-0.322764,3.030866


In [28]:
print(X_train.shape)
print(y_train.shape)

(2353, 56)
(2353,)


In [47]:
test = df[(df.season == 2020)]
X_test = test.drop(['driver', 'podium', 'constructor_points_after_race', 'constructor_wins_after_race', 'constructor_standings_pos_after_race'], axis = 1)
y_test = test.podium

        #scaling
X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

In [48]:
print(X_test.shape)
print(y_test.shape)

(335, 56)
(335,)


In [27]:
X_train.to_csv("X_train.csv")

In [17]:
y_train

17      0
20      0
0       1
1       0
3       0
       ..
2436    0
2433    0
2437    0
2439    0
2424    0
Name: podium, Length: 2353, dtype: int64

In [18]:
comparison_dict ={'model':[],
                  'params': [],
                  'score': []}

In [19]:
# Logistic Regression

params={'penalty': ['l1', 'l2'],
        'solver': ['saga', 'liblinear'],
        'C': np.logspace(-27,1,180)}

for penalty in params['penalty']:
    for solver in params['solver']:
        for c in params['C']:
            model_params = (penalty, solver, c)
            model = LogisticRegression(penalty = penalty, solver = solver, C = c, max_iter = 10000)
            model.fit(X_train, y_train)
            
            model_score = score_classification(model)
            
            comparison_dict['model'].append('logistic_regression')
            comparison_dict['params'].append(model_params)
            comparison_dict['score'].append(model_score)

In [20]:
# Random Forest Classifier

params={'criterion': ['gini', 'entropy'],
        'max_features': [0.8, 'auto', None],
        'max_depth': list(np.linspace(5, 128, 26)) + [None]}

for criterion in params['criterion']:
    for max_features in params['max_features']:
        for max_depth in params['max_depth']:
            model_params = (criterion, max_features, max_depth)
            model = RandomForestClassifier(criterion = criterion, max_features = max_features, max_depth = max_depth)
            model.fit(X_train, y_train)
            
            model_score = score_classification(model)
            
            comparison_dict['model'].append('random_forest_classifier')
            comparison_dict['params'].append(model_params)
            comparison_dict['score'].append(model_score)

In [21]:
# Support Vector Machines

params={'gamma': np.logspace(-4, -1, 20),
        'C': np.logspace(-2, 1, 20),
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid']} 

for gamma in params['gamma']:
    for c in params['C']:
        for kernel in params['kernel']:
            model_params = (gamma, c, kernel)
            model = svm.SVC(probability = True, gamma = gamma, C = c, kernel = kernel )
            model.fit(X_train, y_train)
            
            model_score = score_classification(model)
            
            comparison_dict['model'].append('svm_classifier')
            comparison_dict['params'].append(model_params)
            comparison_dict['score'].append(model_score)

In [22]:
result = pd.DataFrame(comparison_dict).groupby('model')['score'].max()
result
#performs so much better on 2020 than 2021

model
logistic_regression         0.705882
random_forest_classifier    0.764706
svm_classifier              0.647059
Name: score, dtype: float64

In [23]:
results = pd.DataFrame(comparison_dict)

In [52]:
results.sort_values(by='score', ascending = False)[150:180]["params"]

798                                  (gini, None, 123.08)
797                                  (gini, None, 118.16)
1139    (0.00029763514416313193, 0.04281332398719394, ...
1162    (0.00029763514416313193, 0.37926901907322497, ...
1092    (0.00020691380811147902, 0.7847599703514611, rbf)
1141    (0.00029763514416313193, 0.04281332398719394, ...
1163    (0.00029763514416313193, 0.37926901907322497, ...
1140    (0.00029763514416313193, 0.04281332398719394, ...
1160    (0.00029763514416313193, 0.26366508987303583, ...
1138    (0.00029763514416313193, 0.04281332398719394, ...
1137    (0.00029763514416313193, 0.029763514416313176,...
1136    (0.00029763514416313193, 0.029763514416313176,...
1135    (0.00029763514416313193, 0.029763514416313176,...
1093    (0.00020691380811147902, 0.7847599703514611, s...
1134    (0.00029763514416313193, 0.029763514416313176,...
1164    (0.00029763514416313193, 0.37926901907322497, ...
1091    (0.00020691380811147902, 0.7847599703514611, p...
1142    (0.000